In [58]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, PointStruct
from transformers import AutoTokenizer, AutoModelForCausalLM

In [59]:
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [61]:
model_name = 'Qwen/Qwen2.5-7B-Instruct'

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype="auto"
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 100.53it/s]


In [62]:
qdrant = QdrantClient("http://localhost:6333/")
qdrant.create_collection(
    collection_name="quran",
    vectors_config=VectorParams(
        size=768,
        distance="Cosine"
    ),
)

True

In [63]:
quran_translation = pd.read_csv('../../data/quran/translations/english/abdullah-yusuf-ali.csv')
quran_translation.head()

,id,text,foot_notes
0,1,"In the name of Allah, Most Gracious, Most Merc...",NaN
1,2,"Praise be to Allah, the Cherisher and Sustaine...",NaN
2,3,"Most Gracious, Most Merciful;",NaN
3,4,Master of the Day of Judgment.,NaN
4,5,"Thee do we worship, and Thine aid we seek.",NaN


In [64]:
points = []

for id, row in tqdm(quran_translation.iterrows()):
    vector = embedding_model.encode(row['text']).tolist()
    points.append(
        PointStruct(
            id=row['id'],
            vector=vector,
            payload={
                'verse': row['id'],
                'text': row['text'],
            }
        )
    )

6236it [02:57, 35.11it/s]


In [65]:
BATCH_SIZE = 100

for i in range(0, len(points), BATCH_SIZE):
    batch = points[i:i + BATCH_SIZE]
    qdrant.upsert(
        collection_name="quran",
        points=batch
    )

In [74]:
def get_context(question):
    query_vector = embedding_model.encode(question).tolist()

    response = qdrant.query_points(
        collection_name="quran",
        query=query_vector,
        limit=5,
    )

    contexts = []

    for point in response.points:
        print(point)
        contexts.append(point.payload["text"])

    return "\n".join(contexts)

In [80]:
get_context('muhammad')

id=6223 version=63 score=0.5403433 payload={'verse': 6223, 'text': 'Allah, the Eternal, Absolute;'} vector=None shard_key=None order_value=None
id=4903 version=50 score=0.5389828 payload={'verse': 4903, 'text': "It is He Who has taught the Qur'an."} vector=None shard_key=None order_value=None
id=4547 version=46 score=0.53395426 payload={'verse': 4547, 'text': 'But those who believe and work deeds of righteousness, and believe in the (Revelation) sent down to Muhammad - for it is the Truth from their Lord,- He will remove from them their ills and improve their condition.'} vector=None shard_key=None order_value=None
id=5067 version=51 score=0.5315186 payload={'verse': 5067, 'text': 'Thus, then, if he be of those Nearest to Allah,'} vector=None shard_key=None order_value=None
id=1697 version=17 score=0.5264294 payload={'verse': 1697, 'text': '"O my Lord! Thou hast indeed bestowed on me some power, and taught me something of the interpretation of dreams and events,- O Thou Creator of the 

'Allah, the Eternal, Absolute;\nIt is He Who has taught the Qur\'an.\nBut those who believe and work deeds of righteousness, and believe in the (Revelation) sent down to Muhammad - for it is the Truth from their Lord,- He will remove from them their ills and improve their condition.\nThus, then, if he be of those Nearest to Allah,\n"O my Lord! Thou hast indeed bestowed on me some power, and taught me something of the interpretation of dreams and events,- O Thou Creator of the heavens and the earth! Thou art my Protector in this world and in the Hereafter. Take Thou my soul (at death) as one submitting to Thy will (as a Muslim), and unite me with the righteous."'

In [68]:
def generate_answer(question, context):
    prompt = f"""
    You are a question answering system.
    Answer ONLY from the context below.
    If the answer is not present, say:
    "I could not find the answer in the provided text."

    Context:
    {context}

    Question:
    {question}

    Answer:
    """

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=False
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [72]:
def qa_system(question):
    context = get_context(question)
    answer = generate_answer(question, context)

    return {
        "question": question,
        "answer": answer,
    }

In [81]:
response = qa_system("who is our last prophet?")

print("Answer:")
print(response["answer"])

id=4331 version=44 score=0.5984404 payload={'verse': 4331, 'text': 'But how many were the prophets We sent amongst the peoples of old?'} vector=None shard_key=None order_value=None
id=2299 version=23 score=0.5616425 payload={'verse': 2299, 'text': 'When he had turned away from them and from those whom they worshipped besides Allah, We bestowed on him Isaac and Jacob, and each one of them We made a prophet.'} vector=None shard_key=None order_value=None
id=658 version=7 score=0.5392725 payload={'verse': 658, 'text': 'Messengers who gave good news as well as warning, that mankind, after (the coming) of the messengers, should have no plea against Allah: For Allah is Exalted in Power, Wise.'} vector=None shard_key=None order_value=None
id=6223 version=63 score=0.5325858 payload={'verse': 6223, 'text': 'Allah, the Eternal, Absolute;'} vector=None shard_key=None order_value=None
id=4903 version=50 score=0.5256746 payload={'verse': 4903, 'text': "It is He Who has taught the Qur'an."} vector=No